<a href="https://colab.research.google.com/github/Serge3leo/temp-cola/blob/main/stackoverflow-79661744.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to compile the code with current versions of f2py?

[https://stackoverflow.com/questions/79661744/how-to-compile-the-code-with-current-versions-of-f2py](
https://stackoverflow.com/questions/79661744/how-to-compile-the-code-with-current-versions-of-f2py)

In [29]:
try:
    %load_ext fortranmagic
except ModuleNotFoundError:
    # Example of install required packages for know enviroments
    if 'codespace' in str(get_ipython().config):
        !conda install -y -q -c conda-forge fortran-compiler
        %pip install -q -U fortran-magic 'setuptools>=76.1' meson \
                charset-normalizer ninja cmake pkgconfig
        print("WARNING: NEED RESTART")
        %load_ext fortranmagic
    elif 'google.colab' in str(get_ipython().config):
        %pip install -q -U fortran-magic meson \
                charset-normalizer ninja cmake pkgconfig
        %load_ext fortranmagic
        %fortran_config --backend meson -v
    else:
        assert False, """Can't load fortranmagic. For install:
                      pip install -U fortran-magic
                      or
                      conda install -c conda-forge fortran-magic
                      See: https://github.com/mgaitan/fortran_magic/blob/master/documentation.ipynb
                      """

The fortranmagic extension is already loaded. To reload it, use:
  %reload_ext fortranmagic


In [30]:
%%fortran -v
! f2py doesn’t currently support binding interface blocks
! See: https://numpy.org/doc/stable/f2py/advanced/boilerplating.html
!
!      module mod_spatialaverage
!      contains

subroutine spatialaverage(depdata,vardata,depval,varout,sum_a,sum_b,nz,nx,ny)

      implicit none
      real, intent(in) :: depdata(nz,ny,nx)
      real, intent(in) :: vardata(nz,ny,nx)
      integer, intent(in) :: nz,nx,ny
      real, intent(in) :: depval
      real, intent(out) :: varout(ny,nx)
      real, intent(out) :: sum_a(ny,nx),sum_b(ny,nx)
      integer ii,jj,ilev
      real missing_value,dz
      ! ----------------------------------
      varout=0.0;
      missing_value=-1.e3
      ! ----------------------------------
      do ii = 1,nx
        do jj= 1,ny
        ! if it is landpoint:
        if (depdata(1,jj,ii).le.missing_value) cycle
        ! --------------------------------
          dz=abs(0.0-depdata(1,jj,ii));
          sum_a(jj,ii)=sum_a(jj,ii)+vardata(1,jj,ii)*dz
          sum_b(jj,ii)=sum_b(jj,ii)+dz;
          ! --------------------------------
          do ilev=2,nz
            dz=abs(depdata(ilev,jj,ii)-depdata(ilev-1,jj,ii));
            if (depdata(ilev,jj,ii).gt.depval) then
              sum_a(jj,ii)=sum_a(jj,ii)+vardata(ilev,jj,ii)*dz
              sum_b(jj,ii)=sum_b(jj,ii)+dz;
            endif
            if (depdata(ilev,jj,ii).lt.depval.and.depdata(ilev-1,jj,ii).gt.depval) then
              dz=abs(depval-depdata(ilev-1,jj,ii))
              sum_a(jj,ii)=sum_a(jj,ii)+vardata(ilev,jj,ii)*dz
              sum_b(jj,ii)=sum_b(jj,ii)+dz;
            endif
          enddo
        ! --------------------------------
        enddo
      enddo
      varout=sum_a/sum_b;
      ! ==================================
      return
end subroutine spatialaverage

!      end module mod_spatialaverage
      ! ==================================================

The extension _fortran_magic_c7a3dae99f28060a3ec0ff3d7f200600 is already loaded. To reload it, use:
  %fortran_config --clean-cache

Ok. The following fortran objects are ready to use: spatialaverage


In [31]:
print(spatialaverage.__doc__)

varout,sum_a,sum_b = spatialaverage(depdata,vardata,depval,[nz,nx,ny])

Wrapper for ``spatialaverage``.

Parameters
----------
depdata : input rank-3 array('f') with bounds (nz,ny,nx)
vardata : input rank-3 array('f') with bounds (nz,ny,nx)
depval : input float

Other Parameters
----------------
nz : input int, optional
    Default: shape(depdata, 0)
nx : input int, optional
    Default: shape(depdata, 2)
ny : input int, optional
    Default: shape(depdata, 1)

Returns
-------
varout : rank-2 array('f') with bounds (ny,nx)
sum_a : rank-2 array('f') with bounds (ny,nx)
sum_b : rank-2 array('f') with bounds (ny,nx)

